# Photomosaic Preprocessing

The aim of this notebook is to experiment with different images, do color analysis and it will not be very properly structured (although I'll try my best to add comments wherever I can...)

In [163]:
# Import all the necessary libraries and user-defined functions for the task
%load_ext autoreload
%autoreload 2

import os, json
import PIL.Image
import numpy as np
from tqdm.notebook import tqdm

import sys
sys.path.append("../scripts")
from exif_orientation_fix import *
from center_crop import *
from analyse import *

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [164]:
# Define the paths to where your photos are and the source image (one whose mosaic you're interested in making)
photos_path = "../../Photomosaic_Data/Photos/"
source_image_path = "../../Photomosaic_Data/src.jpg"
all_photos = [os.path.join(photos_path, x) for x in os.listdir(photos_path)]

# Define the width to which each photo would be resized
h = 256
IMSIZE = 250

In [165]:
# Fix the orientation of all the photos using piexif
for photo in all_photos:
    rotate_jpeg(photo)

In [166]:
def preprocess_image(image_path, IMSIZE = 250, w = 256):
    # Open the image
    i = PIL.Image.open(image_path)

    # Resize the image by taking aspect ratio into account
    # Keep the width as 256 and calculate height as per the AR
    W, H = i.size
    h = int(H * w / W)
    new_image = i.resize((w, h))

    # Center crop the image to have 250 x 250 standard image size
    center_cropped_image = center_crop(new_image, IMSIZE, IMSIZE)

    return center_cropped_image

In [167]:
# Create a folder to save all the width resized center cropped images
if not os.path.exists("../../Photomosaic_Data/Width_based_crop"):
    os.mkdir("../../Photomosaic_Data/Width_based_crop")

# Use our preprocess function above to first resize based on width and then
# Center crop to 250 x 250 pixels...
for photo in tqdm(all_photos, desc = "Preprocessing Images..."):
    img = preprocess_image(photo)
    photo_name = photo.split("/")[-1].split(".")[0]
    ext = photo.split(".")[-1]
    img.save(f"../../Photomosaic_Data/Width_based_crop/{photo_name}.{ext}")

Preprocessing Images...:   0%|          | 0/34 [00:00<?, ?it/s]

In [168]:
# Make a container to store the photo information
photo_stats = {}

for photo in tqdm(all_photos, desc = "Extracting image stats..."):
    # Get the name of photo from photo path
    photo_name = photo.split("/")[-1]

    # Get the stats for the photo using udf get_image_stats
    ph_stats = get_image_stats(photo)

    # Save the stats against the name of the photo as our index
    photo_stats[photo_name] = ph_stats

with open("../artefacts/photo_stats.json", "w") as f:
    json.dump(photo_stats, f)
    f.close()

Extracting image stats...:   0%|          | 0/34 [00:00<?, ?it/s]